In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from imutils.video import VideoStream
import time
import imutils



Using TensorFlow backend.


ADDITIONAL LOCAL LIBRARIES AND FR MODEL


In [2]:
from fr_utils import *
from inception_blocks_v2 import *
from triplet_loss import *

In [3]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
FRmodel.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "FaceRecoModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 96, 96)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 3, 102, 102)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 48, 48)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 64, 48, 48)   256         conv1[0][0]                      
___

In [4]:

def img_to_encoding_video(image,model):
    img = image[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding
    

FACE DETECTION CASCADE FILE 

In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

COMPILING THE MODEL


In [6]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [7]:
#FRmodel.save("facenet.model")

In [8]:
print("Type 1 if your name is in the database else 0")
x = int(input("Are you registered in database ? "))

Type 1 if your name is in the database else 0
Are you registered in database ? 1


In [9]:
def detect(frame,name): # We create a function that takes as input the image in black and white (gray) and the original image (frame), and that will return the same image with the detector rectangles. 
    faces = face_cascade.detectMultiScale(frame, 1.3, 5) # We apply the detectMultiScale method from the face cascade to locate one or several faces in the image.
    for (x, y, w, h) in faces: # For each detected face:
        frame = frame[x:x+w,y:y+h]
        cv2.imwrite(filename=name+"cropped"+".jpg",img=frame)
    return frame # We return the image with the detector rectangles.


In [10]:
db={}
db["Shreyansh"] = img_to_encoding("Images/crop_shreyansh.jpeg", FRmodel)
db["Kaustav"] = img_to_encoding("Images/crop_kaustav.jpeg", FRmodel)


In [11]:
if(x==0):
    name = input("Enter your name to be stored in the database: ")
    key = cv2.waitKey(1)
    web =cv2.VideoCapture(0)
    while True:
        _,frame = web.read()
        cv2.imshow(name,frame)
        key = cv2.waitKey(1)
        if key == ord('s'):
            cv2.imwrite(filename='saved_img.jpg',img = frame)
            web.release()
            img_new = cv2.imread('saved_img.jpg')
            img_new = cv2.imshow("Captured Image", img_new)
            cv2.waitKey(1650)
            cv2.destroyAllWindows()
            print("Processing image...")
            img_ = cv2.imread('saved_img.jpg')
            img_ = detect(img_,name)
            print("Resizing image to 160x160 scale...")
            img_ = cv2.resize(img_,(96,96))
            img_resized = cv2.imwrite(filename=name+'.jpg', img=img_)
            print("Image saved!")
            db[name]= img_to_encoding(name+'.jpg', FRmodel)
            break
        elif key == ord('q'):
            print("Turning off camera.")
            web.release()
            print("Camera off.")
            print("Program ended.")
            cv2.destroyAllWindows()
            break



In [12]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE ### 
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(image_path,model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding-database[name])

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist<min_dist:
            min_dist = dist
            identity = name

    ### END CODE HERE ###
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [13]:
def who_is_it_video(image, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE ### 
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding_video(image,model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding-database[name])

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist<min_dist:
            min_dist = dist
            identity = name

    ### END CODE HERE ###
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [60]:
image_new= "WhatsApp Image 2020-05-03 at 01.03.18.jpeg"
image_new=cv2.imread(image_new)
image_new=cv2.resize(image_new,(96,96))
who_is_it("oooo.jpg", db, FRmodel)

Not in the database.


(0.8962248, 'Shreyansh')

In [14]:
video_capture= cv2.VideoCapture(0)
while True:
    _,frame = video_capture.read()
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi = frame[y:y+h,x:x+w]
        roi = cv2.resize(roi,(96,96))
        _,db_name= who_is_it_video(roi,db,FRmodel)
        cv2.putText(frame,db_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,255,0), 2)
        cv2.imshow("Frame",frame)
        key= cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break
cv2.destroyAllWindows()
video_capture.release()
        
    

it's Shreyansh, the distance is 0.5595413
it's Shreyansh, the distance is 0.5095657
it's Shreyansh, the distance is 0.52389705
it's Shreyansh, the distance is 0.63940674
it's Shreyansh, the distance is 0.66079766
it's Kaustav, the distance is 0.6534451
it's Kaustav, the distance is 0.5417708
it's Kaustav, the distance is 0.61444575
it's Kaustav, the distance is 0.6420099
it's Kaustav, the distance is 0.6443578
it's Shreyansh, the distance is 0.6606691
it's Kaustav, the distance is 0.66602504
it's Kaustav, the distance is 0.64466304
it's Shreyansh, the distance is 0.5319362
it's Shreyansh, the distance is 0.5833683
it's Shreyansh, the distance is 0.54624504
it's Shreyansh, the distance is 0.69789255
it's Shreyansh, the distance is 0.6119058
it's Kaustav, the distance is 0.67533433
Not in the database.
Not in the database.
Not in the database.
Not in the database.
Not in the database.
Not in the database.
Not in the database.
it's Shreyansh, the distance is 0.63644785
it's Shreyansh, the

it's Kaustav, the distance is 0.5564306
it's Kaustav, the distance is 0.65858454
it's Shreyansh, the distance is 0.65890896
it's Kaustav, the distance is 0.66616946
it's Shreyansh, the distance is 0.62510026
it's Shreyansh, the distance is 0.69071656
Not in the database.
it's Shreyansh, the distance is 0.67684186
Not in the database.
it's Kaustav, the distance is 0.65456915
it's Shreyansh, the distance is 0.6012782
it's Kaustav, the distance is 0.6700136
it's Shreyansh, the distance is 0.6421128
it's Kaustav, the distance is 0.57938606
it's Shreyansh, the distance is 0.64554864
it's Kaustav, the distance is 0.59027356
it's Shreyansh, the distance is 0.54435146
it's Shreyansh, the distance is 0.549087
it's Shreyansh, the distance is 0.6107563
it's Shreyansh, the distance is 0.6260562
it's Shreyansh, the distance is 0.6146748
it's Shreyansh, the distance is 0.5511153
it's Shreyansh, the distance is 0.6484251
it's Kaustav, the distance is 0.58461857
it's Kaustav, the distance is 0.5718336

KeyboardInterrupt: 

In [15]:
cv2.destroyAllWindows()
video_capture.release()

ValueError: Unknown loss function:triplet_loss